## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.model_selection import train_test_split , RandomizedSearchCV , GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import m2cgen
import os
import matplotlib.pyplot as plt
import joblib
from xgboost import XGBClassifier
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

Activity Mapper:

0 : walking
1 : jogging
2 : upstairs
3 : downstairs
4 : sitting
5 : standing
6 : sleeping

## Load Data

In [2]:
dataset_path = os.getcwd().replace('/notebook' , '') + "/dataset/"

# Read training data
imu_df = pd.read_csv(dataset_path + "imu_data_v2.csv")
imu_df = imu_df.iloc[:,1:]
imu_df.columns = imu_df.columns.str.replace("'" , "")
x = imu_df.iloc[:,:-1]
y = imu_df.iloc[:,-1]

In [3]:
x.columns

Index(['attitude.roll', ' attitude.pitch', ' attitude.yaw', ' gravity.x', ' gravity.y', ' gravity.z', ' rotationRate.x', ' rotationRate.y', ' rotationRate.z', ' userAcceleration.x', ' userAcceleration.y', ' userAcceleration.z', ' magnetic.x', ' magnetic.y', ' magnetic.z', ' temperature'], dtype='object')

# Modeling

## Motion Classification

In [4]:
"""
Map
0 - steady motion,
1 - unsteady motion
"""
mapper = {
    0 : 1,
    1 : 1,
    2 : 1,
    3 : 1,
    4 : 0,
    5 : 0,
    6 : 0
}

In [5]:
motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [7]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [2 , 3 , 4 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
11,0.998194,"{'criterion': 'entropy', 'max_depth': 10}"
10,0.997760,"{'criterion': 'entropy', 'max_depth': 8}"
5,0.997101,"{'criterion': 'gini', 'max_depth': 10}"
4,0.996806,"{'criterion': 'gini', 'max_depth': 8}"
9,0.996372,"{'criterion': 'entropy', 'max_depth': 6}"
3,0.994861,"{'criterion': 'gini', 'max_depth': 6}"
8,0.992135,"{'criterion': 'entropy', 'max_depth': 4}"
2,0.981406,"{'criterion': 'gini', 'max_depth': 4}"
7,0.967413,"{'criterion': 'entropy', 'max_depth': 3}"
1,0.963125,"{'criterion': 'gini', 'max_depth': 3}"


In [8]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'entropy')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6719
           1       1.00      1.00      1.00      7681

    accuracy                           1.00     14400
   macro avg       1.00      1.00      1.00     14400
weighted avg       1.00      1.00      1.00     14400



In [9]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     26884
           1       1.00      1.00      1.00     30716

    accuracy                           1.00     57600
   macro avg       1.00      1.00      1.00     57600
weighted avg       1.00      1.00      1.00     57600



In [10]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8402
           1       1.00      1.00      1.00      9598

    accuracy                           1.00     18000
   macro avg       1.00      1.00      1.00     18000
weighted avg       1.00      1.00      1.00     18000



In [11]:
with open(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" ))

13383


## Steady Movement Classification

In [12]:
steady_df  =imu_df[(imu_df['Target'] == 4) | (imu_df['Target'] == 5) | (imu_df['Target'] == 6)]
x = steady_df.iloc[:,:-1]
y = steady_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [14]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [3 , 4 , 5 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
11,0.990253,"{'criterion': 'entropy', 'max_depth': 10}"
5,0.986161,"{'criterion': 'gini', 'max_depth': 10}"
4,0.980878,"{'criterion': 'gini', 'max_depth': 8}"
10,0.980283,"{'criterion': 'entropy', 'max_depth': 8}"
3,0.967001,"{'criterion': 'gini', 'max_depth': 6}"
9,0.965699,"{'criterion': 'entropy', 'max_depth': 6}"
2,0.960603,"{'criterion': 'gini', 'max_depth': 5}"
8,0.951749,"{'criterion': 'entropy', 'max_depth': 5}"
1,0.904018,"{'criterion': 'gini', 'max_depth': 4}"
7,0.864583,"{'criterion': 'entropy', 'max_depth': 4}"


In [17]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'entropy')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           4       0.98      0.98      0.98      2241
           5       0.98      0.98      0.98      2239
           6       1.00      1.00      1.00      2240

    accuracy                           0.99      6720
   macro avg       0.99      0.99      0.99      6720
weighted avg       0.99      0.99      0.99      6720



In [18]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           4       0.99      0.99      0.99      8934
           5       0.99      0.99      0.99      8986
           6       1.00      1.00      1.00      8960

    accuracy                           0.99     26880
   macro avg       0.99      0.99      0.99     26880
weighted avg       0.99      0.99      0.99     26880



In [19]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           4       0.98      0.99      0.99      2790
           5       0.99      0.98      0.99      2810
           6       1.00      1.00      1.00      2800

    accuracy                           0.99      8400
   macro avg       0.99      0.99      0.99      8400
weighted avg       0.99      0.99      0.99      8400



In [20]:
with open(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" ))

9468


## Unsteady Movement Classification

In [21]:
unsteady_df  =imu_df[(imu_df['Target'] != 4) & (imu_df['Target'] != 5) & (imu_df['Target'] != 6)]
x = unsteady_df.iloc[:,:-1]
y = unsteady_df.iloc[:,-1]

## Unsteady Motion Classification

In [22]:
"""
Map
0 - surface motion,
1 - staircase motion
"""
mapper = {
    0 : 0,
    1 : 0,
    2 : 1,
    3 : 1
}

In [23]:
unsteady_motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,unsteady_motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [24]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
5,0.995020,"{'criterion': 'gini', 'max_depth': 10}"
11,0.994564,"{'criterion': 'entropy', 'max_depth': 10}"
4,0.994141,"{'criterion': 'gini', 'max_depth': 8}"
10,0.993750,"{'criterion': 'entropy', 'max_depth': 8}"
9,0.992220,"{'criterion': 'entropy', 'max_depth': 7}"
3,0.991341,"{'criterion': 'gini', 'max_depth': 7}"
8,0.990820,"{'criterion': 'entropy', 'max_depth': 6}"
2,0.989941,"{'criterion': 'gini', 'max_depth': 6}"
7,0.987891,"{'criterion': 'entropy', 'max_depth': 5}"
1,0.984896,"{'criterion': 'gini', 'max_depth': 5}"


In [25]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'entropy')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3847
           1       1.00      1.00      1.00      3833

    accuracy                           1.00      7680
   macro avg       1.00      1.00      1.00      7680
weighted avg       1.00      1.00      1.00      7680



In [26]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15394
           1       1.00      1.00      1.00     15326

    accuracy                           1.00     30720
   macro avg       1.00      1.00      1.00     30720
weighted avg       1.00      1.00      1.00     30720



In [27]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4810
           1       0.99      1.00      0.99      4790

    accuracy                           0.99      9600
   macro avg       0.99      0.99      0.99      9600
weighted avg       0.99      0.99      0.99      9600



In [28]:
with open(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" ))

15741


## Staircase motion

In [29]:
staircase_df  =imu_df[(imu_df['Target'] == 1) | (imu_df['Target'] == 2)]
x = staircase_df.iloc[:,:-1]
y = staircase_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [30]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
10,0.990755,"{'criterion': 'entropy', 'max_depth': 8}"
11,0.990560,"{'criterion': 'entropy', 'max_depth': 10}"
9,0.989388,"{'criterion': 'entropy', 'max_depth': 7}"
8,0.989323,"{'criterion': 'entropy', 'max_depth': 6}"
5,0.988867,"{'criterion': 'gini', 'max_depth': 10}"
4,0.988672,"{'criterion': 'gini', 'max_depth': 8}"
3,0.985482,"{'criterion': 'gini', 'max_depth': 7}"
2,0.968099,"{'criterion': 'gini', 'max_depth': 6}"
7,0.963216,"{'criterion': 'entropy', 'max_depth': 5}"
6,0.959766,"{'criterion': 'entropy', 'max_depth': 4}"


In [31]:
dt = DecisionTreeClassifier(max_depth = 8 , criterion= 'entropy')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           1       0.99      0.99      0.99      1922
           2       0.99      0.99      0.99      1918

    accuracy                           0.99      3840
   macro avg       0.99      0.99      0.99      3840
weighted avg       0.99      0.99      0.99      3840



In [32]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           1       1.00      0.99      1.00      7715
           2       0.99      1.00      1.00      7645

    accuracy                           1.00     15360
   macro avg       1.00      1.00      1.00     15360
weighted avg       1.00      1.00      1.00     15360



In [33]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           1       1.00      0.99      0.99      2408
           2       0.99      1.00      0.99      2392

    accuracy                           0.99      4800
   macro avg       0.99      0.99      0.99      4800
weighted avg       0.99      0.99      0.99      4800



In [34]:
with open(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" ))

9242


## Surface motion classification

In [35]:
surface_df  =imu_df[(imu_df['Target'] == 0) | (imu_df['Target'] == 4)]
x = surface_df.iloc[:,:-1]
y = surface_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [36]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
11,0.999639,"{'criterion': 'entropy', 'max_depth': 10}"
4,0.999519,"{'criterion': 'gini', 'max_depth': 8}"
6,0.999519,"{'criterion': 'entropy', 'max_depth': 4}"
7,0.999519,"{'criterion': 'entropy', 'max_depth': 5}"
9,0.999519,"{'criterion': 'entropy', 'max_depth': 7}"
1,0.999459,"{'criterion': 'gini', 'max_depth': 5}"
3,0.999399,"{'criterion': 'gini', 'max_depth': 7}"
10,0.999339,"{'criterion': 'entropy', 'max_depth': 8}"
8,0.999279,"{'criterion': 'entropy', 'max_depth': 6}"
2,0.999159,"{'criterion': 'gini', 'max_depth': 6}"


In [37]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'entropy')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1918
           4       1.00      1.00      1.00      2242

    accuracy                           1.00      4160
   macro avg       1.00      1.00      1.00      4160
weighted avg       1.00      1.00      1.00      4160



In [38]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7680
           4       1.00      1.00      1.00      8960

    accuracy                           1.00     16640
   macro avg       1.00      1.00      1.00     16640
weighted avg       1.00      1.00      1.00     16640



In [39]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2399
           4       1.00      1.00      1.00      2801

    accuracy                           1.00      5200
   macro avg       1.00      1.00      1.00      5200
weighted avg       1.00      1.00      1.00      5200



In [40]:
with open(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" ))

1377
